# Parallel multi-point sequential Bayesian optimisation

In [1]:
import torch
from nubo.acquisition import MCExpectedImprovement, MCUpperConfidenceBound
from nubo.models import GaussianProcess, fit_gp
from nubo.optimisation import sequential
from nubo.test_functions import Hartmann6D
from nubo.utils import gen_inputs
from gpytorch.likelihoods import GaussianLikelihood


# test function
func = Hartmann6D(minimise=False)
dims = func.dims
bounds = func.bounds

# training data
x_train = gen_inputs(num_points=dims*5,
                     num_dims=dims,
                     bounds=bounds)
y_train = func(x_train)

# Bayesian optimisation loop
iters = 10

for iter in range(iters):
    
    # specify Gaussian process
    likelihood = GaussianLikelihood()
    gp = GaussianProcess(x_train, y_train, likelihood=likelihood)
    
    # fit Gaussian process
    fit_gp(x_train, y_train, gp=gp, likelihood=likelihood, lr=0.1, steps=200)

    # specify acquisition function
    # acq = MCExpectedImprovement(gp=gp, y_best=torch.max(y_train), samples=256)
    acq = MCUpperConfidenceBound(gp=gp, beta=1.96**2, samples=512)

    # optimise acquisition function
    x_new, _ = sequential(func=acq, method="Adam", batch_size=4, bounds=bounds, lr=0.1, steps=200, num_starts=1)

    # evaluate new point
    y_new = func(x_new)
    
    # add to data
    x_train = torch.vstack((x_train, x_new))
    y_train = torch.hstack((y_train, y_new))

    # print new best
    if torch.max(y_new) > torch.max(y_train[:-y_new.size(0)]):
        best_eval = torch.argmax(y_train)
        print(f"New best at evaluation {best_eval+1}: \t Inputs: {x_train[best_eval, :].numpy().reshape(dims).round(4)}, \t Outputs: {-y_train[best_eval].numpy().round(4)}")

# results
best_iter = int(torch.argmax(y_train))
print(f"Evaluation: {best_iter+1} \t Solution: {float(y_train[best_iter]):.4f}")


New best at evaluation 35: 	 Inputs: [0.3297 0.0108 0.1252 0.1826 0.1695 0.7313], 	 Outputs: -1.5985
New best at evaluation 39: 	 Inputs: [0.0422 0.0106 0.0614 0.2066 0.2079 0.749 ], 	 Outputs: -1.6499
New best at evaluation 43: 	 Inputs: [0.2429 0.0024 0.1681 0.235  0.196  0.6835], 	 Outputs: -2.1151
New best at evaluation 47: 	 Inputs: [0.257  0.0045 0.2255 0.2424 0.2303 0.6589], 	 Outputs: -2.4768
New best at evaluation 51: 	 Inputs: [0.2588 0.0019 0.3118 0.2619 0.2567 0.6526], 	 Outputs: -2.777
New best at evaluation 59: 	 Inputs: [0.2349 0.0065 0.3678 0.3052 0.2844 0.6723], 	 Outputs: -2.9674
New best at evaluation 67: 	 Inputs: [0.2252 0.0888 0.367  0.3118 0.2958 0.6721], 	 Outputs: -3.1527
Evaluation: 67 	 Solution: 3.1527
